In [91]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
import re


In [92]:
data = pd.read_csv('tweets2.csv')
# Keeping only the neccessary columns
data.dtypes.index
data = data[['text', 'tweet_class']]

#removing all Junk tweets
# data = data[ data.tweet_class != 2 ]

In [93]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [94]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


/home/clasick/Documents/hackathon/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
/home/clasick/Documents/hackathon/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 54, 128)           256000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 788       
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________
None


In [95]:
df = data.copy()
df.loc[df.tweet_class == -1, 'tweet_class'] = 3

Y = df['tweet_class']

# encoder = LabelBinarizer()
# transformed_Y = encoder.fit_transform(Y)

from keras.utils import to_categorical
transformed_Y = to_categorical(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X,transformed_Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(468, 54) (468, 4)
(231, 54) (231, 4)


In [96]:
print(X_test.shape)
print(X_train.shape)
print(Y_test.shape)
print(Y_train.shape)

(231, 54)
(468, 54)
(231, 4)
(468, 4)


In [97]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 2s - loss: 1.0461 - acc: 0.7521
Epoch 2/7
 - 1s - loss: 0.7234 - acc: 0.7991
Epoch 3/7
 - 1s - loss: 0.6790 - acc: 0.7991
Epoch 4/7
 - 1s - loss: 0.6125 - acc: 0.7991
Epoch 5/7
 - 1s - loss: 0.5074 - acc: 0.8120
Epoch 6/7
 - 1s - loss: 0.3945 - acc: 0.8547
Epoch 7/7
 - 1s - loss: 0.3222 - acc: 0.8803


In [99]:
validation_size = 50

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.75
acc: 0.75


In [105]:
my_test = tokenizer.texts_to_sequences(['am i the only one who hates the walking dead'])

print(my_test) 

test1 = numpy.asarray(my_test)

test1 = pad_sequences(test1, maxlen=54)

test1

[[177, 7, 1, 108, 55, 91, 710, 1, 4, 3]]


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 177,   7,   1, 108,  55,  91, 710,   1,
          4,   3]], dtype=int32)

In [107]:
print(model.predict_classes(test1))

[2]
